# Simple Explicit Baseline
* Computes a bias for each user and for each item
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = m + u_i + a_j$
* $m = \text{mean}_{ij}(r_{ij})$
* $u_i = \text{mean}_j(r_{ij}) - m$
* $a_j = \text{mean}_i(r_{ij}) - m$
* $r_{ij}$ is the rating for user $i$ and item $j$
* Useful as a benchmark to compare more sophicated algorithms against

In [1]:
const name = "SimpleExplicitBaseline"
const residual_alphas = []
const implicit = false;

In [2]:
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb");

In [3]:
using DataFrames
import DataStructures: DefaultDict
import Statistics: mean

## Training

In [4]:
function get_residualized_dataframe(split, implicit, residual_alphas)
    df = get_split(split, implicit)
    residuals = read_alpha(residual_alphas, split, implicit)
    DataFrame(user = df.user, item = df.item, rating = df.rating - residuals.rating)
end

const training = get_residualized_dataframe("training", implicit, residual_alphas);

In [5]:
const μ = mean(training.rating);

In [6]:
u = combine(groupby(training, :user), :rating => mean => :rating)
u = Dict(Pair.(u.user, u.rating .- μ));

In [7]:
a = combine(groupby(training, :item), :rating => mean => :rating)
a = Dict(Pair.(a.item, a.rating .- μ));

## Inference

In [8]:
function make_prediction(users, items, u, a, μ)
    u = DefaultDict(zero(eltype(μ)), u)
    a = DefaultDict(zero(eltype(μ)), a)
    r = Array{eltype(μ)}(undef, length(users))
    Threads.@threads for i = 1:length(r)
        r[i] = u[users[i]] + a[items[i]] + μ
    end
    r
end;

model(users, items) = make_prediction(users, items, u, a, μ);

In [9]:
write_alpha(model, residual_alphas, implicit, name);

[ Info: 20220621 20:52:49 validation loss: 1.9053539, β: Float32[0.9762136]
Progress: 100%|███████████████████████████| Time: 0:00:00 (27.81 ns/it)
[ Info: 20220621 20:52:51 training loss: 1.6972578, β: Float32[0.9762136]


In [10]:
write_params(Dict("u" => u, "a" => a, "μ" => μ), name);